In [1]:
import numpy as np
from sklearn.datasets import load_digits
from sklearn.manifold import TSNE
from tqdm.notebook import tqdm
from numba import njit
from functools import lru_cache

In [2]:
import matplotlib.pyplot as plt
from matplotlib import rcParams
from matplotlib.patches import Patch
from matplotlib.colors import ListedColormap, LinearSegmentedColormap

In [3]:
plt.style.use('seaborn-whitegrid')
rcParams['font.size'] = 18
rcParams['figure.figsize'] = (12, 8)

In [4]:
def squared_dist_mat(X):
    sum_X = np.sum(np.square(X), 1)
    D = np.add(np.add(-2 * np.dot(X, X.T), sum_X).T, sum_X)
    return D

In [5]:
def compute_partial(i, P, Q, Y):
    # eq 5
    m,n = Y.shape
    partial = np.zeros(n)
    # TODO: vectorize
    for j in range(m):
        partial+=(P[i,j] - Q[i,j]) * (Y[i,:] - Y[j,:]) * (1 + np.linalg.norm(Y[i,:] - Y[j,:])**2)**(-1)
    
    return 4*partial

def compute_grad(P, Q, Y):
    # eq 5
    m = Y.shape[0]
    grad = np.zeros_like(Y)
    for i in range(m):
        grad[i,:] = compute_partial(i, P, Q, Y)
    return grad

In [19]:
def compute_grad_vector(P, Q, Y):
    dist_mat = squared_dist_mat(Y)
    Ydiff = (Y[:, np.newaxis, :] - Y[np.newaxis, :, :])
    pq_factor = (P-Q)[:, :, np.newaxis]
    dist_factor = ((1+dist_mat)**(-1))[:, :, np.newaxis]
    return np.sum(4*pq_factor*Ydiff*dist_factor, axis=1)

In [20]:
P = np.random.randn(100, 100)
Q = np.random.randn(100, 100)
Y = np.random.randn(100, 2)

In [21]:
Grad_old = compute_grad(P, Q, Y)
Grad_new = compute_grad_vector(P, Q, Y)
np.allclose(Grad_old, Grad_new)

(100, 2)


True

In [22]:
%%timeit -n10 -r5
Grad_old = compute_grad(P, Q, Y)

121 ms ± 3.34 ms per loop (mean ± std. dev. of 5 runs, 10 loops each)


In [23]:
%%timeit -n10 -r5
Grad_new = compute_grad_vector(P, Q, Y)

663 µs ± 158 µs per loop (mean ± std. dev. of 5 runs, 10 loops each)
